<h1><center><font size=5><font color='gray'>**Word2vec**</font></font></center></h1>

### Importing Dependencies

In [26]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re
import numpy as np
import nltk
from gensim.models import Word2Vec
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from nltk.tokenize import WordPunctTokenizer
from collections import Counter
from string import punctuation, ascii_lowercase
import re
from tqdm import tqdm

In [27]:
% cd "C:\Users\vibhanshuv\Downloads\Case Study 1\aclImdb"

C:\Users\vibhanshuv\Downloads\Case Study 1\aclImdb


### Importing csv files

In [28]:
train_df=pd.read_csv('pre_train.csv',sep=",")
test_df=pd.read_csv('pre_test.csv',sep=",")

### Shuffling the order of dataframes for better training

In [29]:
train_df=train_df.reindex(np.random.permutation(train_df.index))
test_df=test_df.reindex(np.random.permutation(test_df.index))

In [30]:
train_df.head()

,Unnamed: 0,description,sentiment,words,words_sent
19844,19844,amongst the standard one liner type action fil...,1,"['amongst', 'standard', 'one', 'liner', 'type'...",amongst standard one liner type action film ac...
2479,2479,"i saw this movie, and at times, i was unnerved...",0,"['saw', 'movie', 'time', 'unnerved', 'believin...",saw movie time unnerved believing movie 'saw m...
19784,19784,saw this again recently on comedy central. i w...,1,"['saw', 'again', 'recently', 'comedy', 'centra...",saw again recently comedy central would love s...
22950,22950,"after a big tip of the hat to spinal tap, this...",1,"['big', 'tip', 'hat', 'spinal', 'tap', 'movie'...",big tip hat spinal tap movie hilarious anyone ...
11029,11029,"after watching ""the bodyguard"" last night, i f...",0,"['watching', '``', 'bodyguard', ""''"", 'last', ...",watching `` bodyguard '' last night felt compe...


#### Splitting the labelled data

In [31]:
Y_train=train_df['sentiment']
Y_test=test_df['sentiment']

### Setup Tokenizer

In [32]:
tokenizer = WordPunctTokenizer()
vocab = Counter()

In [33]:
def text_to_wordlist(text, lower=False):
  
    # Tokenize
    text = tokenizer.tokenize(text)
    
    # optional: lower case
    if lower:
        text = [t.lower() for t in text]
    
    # Return a list of words
    vocab.update(text)
    return text

In [34]:
def process_comments(list_sentences, lower=False):
    comments = []
    for text in tqdm(list_sentences):
        txt = text_to_wordlist(text, lower=lower)
        comments.append(txt)
    return comments

In [35]:
list_sentences_train = list(train_df["words_sent"].values)
list_sentences_test = list(test_df["words_sent"].values)

In [36]:
comments = process_comments(list_sentences_train + list_sentences_test, lower=True)

100%|██████████████████████████████████████████████████████████████████████████| 50000/50000 [00:05<00:00, 9940.90it/s]


In [37]:
print("The vocabulary contains {} unique tokens".format(len(vocab)))

The vocabulary contains 98201 unique tokens


In [38]:
model = Word2Vec(comments, window=5, size=5000,min_count=5, workers=16, sg=0, negative=5)

In [39]:
word_vectors = model.wv

In [40]:
def w2v_preprocessing(df):
    """ All the preprocessing steps for word2vec are done in this function.
    All mutations are done on the dataframe itself. So this function returns
    nothing.
    """
    df['text'] = df.description.str.lower()
    df['document_sentences'] = df.text.str.split('.')  # split texts into individual sentences
    df['tokenized_sentences'] = list(map(lambda sentences:
                                         list(map(nltk.word_tokenize, sentences)),
                                         df.document_sentences))  # tokenize sentences
    df['tokenized_sentences'] = list(map(lambda sentences:
                                         list(filter(lambda lst: lst, sentences)),
                                         df.tokenized_sentences))  # remove empty lists

In [41]:
w2v_preprocessing(train_df)

w2v_preprocessing(test_df)

In [42]:
def get_w2v_features(w2v_model, sentence_group):
    """ Transform a sentence_group (containing multiple lists
    of words) into a feature vector. It averages out all the
    word vectors of the sentence_group.
    """
    words = np.concatenate(sentence_group)  # words in text
    index2word_set = set(w2v_model.wv.vocab.keys())  # words known to model
    
    featureVec = np.zeros(w2v_model.vector_size, dtype="float32")
    
    # Initialize a counter for number of words in a review
    nwords = 0
    # Loop over each word in the comment and, if it is in the model's vocabulary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            featureVec = np.add(featureVec, w2v_model[word])
            nwords += 1.

    # Divide the result by the number of words to get the average
    if nwords > 0:
        featureVec = np.divide(featureVec, nwords)
    return featureVec

In [43]:
train_df['w2v_features'] = list(map(lambda sen_group:
                                      get_w2v_features(model, sen_group),
                                      train_df.tokenized_sentences))

test_df['w2v_features'] = list(map(lambda sen_group:
                                      get_w2v_features(model, sen_group),
                                      test_df.tokenized_sentences))

C:\Users\vibhanshuv\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [44]:
X_train_w2v = np.array(list(map(np.array, train_df.w2v_features)))    

X_test_w2v = np.array(list(map(np.array, test_df.w2v_features)))   

In [48]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0)
#from sklearn.model_selection import GridSearchCV
#parameters={'C':[1.0,10.0,12.0,20,50,100]}
#clf=GridSearchCV(model,param_grid=parameters,cv=3)
clf.fit(X_train_w2v,Y_train)
#clf.best_params_
#print('best score :{0:.2F}'.format(clf.best_score_))
clf.score(X_test_w2v,Y_test)

0.86528

In [49]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
model.fit(X_train_w2v,Y_train)
model.score(X_test_w2v,Y_test)

0.84616

In [51]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=0)
from sklearn.model_selection import GridSearchCV
parameters={'C':[1.0,10.0,12.0,20,50,100]}
clf=GridSearchCV(model,param_grid=parameters,cv=3)
clf.fit(X_train_w2v,Y_train)
clf.best_params_
print('best score :{0:.2F}'.format(clf.best_score_))
clf.score(X_test_w2v,Y_test)

best score :0.87


0.87256

In [52]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=100)
model.fit(X_train_w2v,Y_train)
model.score(X_test_w2v,Y_test)

0.83592

In [45]:
from sklearn.naive_bayes import BernoulliNB
classifier=BernoulliNB().fit(X_train_w2v,Y_train)
classifier.score(X_test_w2v,Y_test)

0.71016

In [47]:
from sklearn import linear_model
clf = linear_model.SGDClassifier(max_iter=1000)
clf.fit(X_train_w2v,Y_train)
clf.score(X_test_w2v,Y_test)

0.86332